#### Classroom metrics analysis

In [ ]:
from PIL import Image, ImageDraw
from datetime import date
from io import BytesIO
from pandas import DataFrame
from scipy import stats
from sklearn import preprocessing
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sshtunnel import SSHTunnelForwarder
import base64
import json
import math
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint
import pymongo
import seaborn as sns; sns.set()
import statistics

In [ ]:
print("Connecting to MongoDB...")

MONGO_HOST = ""
MONGO_DB = ""
MONGO_USER = ""
MONGO_PASS = ""

server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_username=MONGO_USER,
    ssh_password=MONGO_PASS,
    remote_bind_address=('127.0.0.1', 27017)
)

server.start()
# local port connection
client = pymongo.MongoClient('127.0.0.1', server.local_bind_port)
db = client[MONGO_DB]
print("\033[92mConnection successful\033[0m")

collection = db['']

In [ ]:
def stuEngagement(video_collections):
  all_arms_cross = []
  all_hands_face = []
  all_hands_raised = []

  for v_col in video_collections:
      collection = db[v_col]
      arms_cross = 0
      hands_on_face = 0
      hands_raised = 0
      frames = collection.find({"channel": "student"})

      for frame in frames:
          frame_num = frame["frameNumber"]
          num_errors = 0
          try:
              for person in frame["people"]:
                  if person["inference"]["posture"]["armPose"] == "armsCrossed":
                      arms_cross += 1
                  if person["inference"]["posture"]["armPose"] == "handsOnFace":
                      hands_on_face +=1
                  if person["inference"]["posture"]["armPose"] == "handsRaised":
                      hands_raised +=1
          except:
              num_errors += 1   


      all_arms_cross.append(arms_cross)
      all_hands_face.append(hands_on_face)
      all_hands_raised.append(hands_raised)

  return (all_arms_cross, all_hands_face, all_hands_raised)

In [ ]:
def attendance(video_collections):
  all_attendances = []

  for v_col in video_collections:
      collection = db[v_col]
      attendance = 0
      frame_maxstus = 0
      frames = collection.find({"channel": "student"})

      for frame in frames:
          frame_num = frame["frameNumber"]
          num_errors = 0
          try:
              num_stus = len(frame["people"])
              if(num_stus > attendance):
                  attendance = num_stus
                  frame_maxstus = frame_num
          except:
              num_errors += 1   
      all_attendances.append(attendance)

  return all_attendances

In [ ]:
def simpleTrends():
  # compute simple metrics using regression analysis 
  metrics_insights = []
  metrics_insights.append(str(statistics.median(all_attendances)))
  metrics_insights.append(str(round(statistics.mean(all_hands_raised), 2)))

  movement_insights = []
  rec_ind = list(range(len(all_attendances)))
  coeffs = np.polyfit(rec_ind, all_mov_stdev, 1)
  slope = coeffs[-2]
  if slope < -0.5:
      movement_insights.append("mtd")
  elif slope > 0.5:
      movement_insights.append("mtu")
  else:
      movement_insights.append("mtc")
      
  gaze_insights = []
  rec_ind = list(range(len(all_attendances)))
  coeffs = np.polyfit(rec_ind, all_gaze_stdev, 1)
  slope = coeffs[-2]
  if slope < -0.5:
      gaze_insights.append("gtd")
  elif slope > 0.5:
      gaze_insights.append("gtu")
  else:
      gaze_insights.append("gtc")

  return (metrics_insights, movement_insights, gaze_insights)